In [ ]:
from fastai.vision import *
from pathlib import Path

In [ ]:
path = Path(r"X:\data\net-detect\data")

In [ ]:
data = ImageDataBunch.from_folder(path, train="train", ds_tfms=get_transforms(do_flip=False, max_zoom=1.0), size=(144,255), valid_pct=0.2).normalize(imagenet_stats)
data.show_batch(rows=4)

In [ ]:
learn = cnn_learner(data, models.resnet18, metrics=accuracy)


In [ ]:
learn.fit_one_cycle(10)
learn.save("x:/cm/nd-10-resnet18-prefine-noflip")

In [ ]:
learn.fit_one_cycle(10)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save("x:/cm/model/nd-10-resnet18-prefine")


In [ ]:
learn = learn.load("x:/cm/nd-10-resnet18-prefine")

In [ ]:
learn.export("x:/models/net-detect-resnet18.pkl")

In [ ]:
learn.unfreeze()

learn.fit_one_cycle(3)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save("x:/cm/model/nd-10-resnet-18-refined-noflip")

In [ ]:
learn = learn.load("x:/cm/model/nd-10-resnet-18-refined-noflip")

test_data = ImageList.from_folder(path / "test").split_none().label_from_folder().databunch().normalize(imagenet_stats)



In [ ]:
m = learn.model.eval()

In [ ]:
from fastai.callbacks.hooks import *

def hooked_backward(cat):
    with hook_output(m[0]) as hook_a: 
        with hook_output(m[0], grad=True) as hook_g:
            preds = m(xb)
            preds[0,int(cat)].backward()
    return hook_a,hook_g


def show_heatmap(hm):
    plt.figure()
    _,ax = plt.subplots()
    xb_im.show(ax)
    ax.imshow(hm, alpha=0.6, extent=(0,256,144,0),
              interpolation='bilinear', cmap='magma')
    plt.show()



In [ ]:
predictions = []

for x, y in test_data.train_ds:
    xb,_ = test_data.one_item(x)
    xb_im = Image(test_data.denorm(xb)[0])
    xb = xb.cuda()

    hook_a,hook_g = hooked_backward(y)
    acts  = hook_a.stored[0].cpu()
    acts.shape

    avg_acts = acts.mean(0)
    avg_acts.shape
    pred = learn.predict(x)
    print("actual:", y, "predicted:", pred[0])
    predictions.append({"y": y, "y_": pred})
    show_heatmap(avg_acts)



In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import functools
lookup = {"left": 0, "right": 2, "none": 0}

y_true = list(map(lambda _: lookup[str(_["y"])], predictions))
y_pred = list(map(lambda _:lookup[str(_["y_"][0])], predictions))
sum(list(map(int, map(lambda _: _[0] == _[1], zip(y_true, y_pred))))), len(y_pred)

In [ ]:
predictions[200]

In [ ]:
data.classes